# I. Introduction

In this notebook, we will implement genetic algorithm (**GA**) to solve the n-Queens problem (find an arrangement for n queens on nxn board so that no queen can attack the others). For illustration, we will use 8x8 board and here is an example of solution

![8q_solution](example_solution.jpg)

First we represent a board configuration as a sequence of each queen's position in each column so the above board can be represented by 

$$
(3, 1, 7, 5, 8, 2, 4, 6)
$$

Next, we need to define a fitness function, e.g for 8-queens problem, we can use
$$
\texttt{fitness}(board) = 28 - \#\text{attacking pairs}
$$
where $\texttt{fitness}(board)=28$ means no attacking pairs i.e a solution (note that 28 is the maximum number of attacking pairs). So to solve the n-queen is equivalent to maximize the fitness function.

The GA contains the following steps
1. **Initialization**: we generate randomly a distributed population (we control the number of samples)
2. **Selection of parents**: using the fitness function we define the probability of a parent is selected as
$$
\mathbb{P}(p_i \text{ is selected}) = \frac{\texttt{fitness}(p_i)}{\sum_i\texttt{fitness}(p_i)}
$$
3. **CrossOver**: once a pair of parents is selected, the crossover/reproduction is illustrated as in the following image
![crossover](crossover.png)
where the parents are $(3,2,7,5,2,4,1,1)$ and $(2,4,7,4,8,5,5,2)$, we randomly select where to cut the two sequences. Then the first child is the first part of parent 1 and the second part of parent 2, the second child is the first part of parent 2 with the second part of parent 1.
4. **Mutation**: for each digit in the sequence we have a small but signficant chance that it will mutate into some other digit as illustrated below
![mutation](mutation.png)
here the first child has one mutated digit $5\rightarrow1$. 

This mimics evolution in biology, the main idea is to handle the case when a critical piece doesn't appears in one stage (e.g at some stage where all board in the population doesn't has a digit then any next generation won't have this digit which makes impossible to find a solution).

Let's implement **GA**

# II. Representing the board
In order to create a population, we need to build a representation of the board

In [32]:
import numpy as np

def is_attacking(loc1, loc2):
    if (loc1[1] == loc2[1]) or (loc1[0] + loc1[1] == loc2[0] + loc2[1]) \
        or (loc1[0] - loc1[1] == loc2[0] - loc2[1]):
        return True
    else:
        return False

class Board:
    def __init__(self, n, sequences=None):
        self._n = n
        self._high = n + 1
        self._sequences = sequences
        if self._sequences is None:
            self._sequences = np.random.randint(1, high=self._high, size=(n))
        
        self._max_ap = self._n * (self._n - 1) // 2
    
    @property
    def length(self):
        return self._n
    
    @property
    def sequences(self):
        return self._sequences
    
    def fitness(self):
        fit = self._max_ap
        for i in range(self._n-1):
            for j in range(i+1, self._n):
                if is_attacking((i, self._sequences[i]), (j, self._sequences[j])):
                    fit -= 1
        return fit
    
    def mutate(self, i):
        self._sequences[i] = np.random.randint(1, high=self._high)

Let's try it with some example (taken for Udacity's lecture)

In [21]:
b1 = Board(8, np.array([3,2,7,4,8,5,5,2]))
b2 = Board(8, np.array([3,2,7,5,2,1,2,4]))
b3 = Board(8, np.array([2,4,7,5,2,4,1,1]))
print('b1 has fitness {}'.format(b1.fitness()))
print('b2 has fitness {}'.format(b2.fitness()))
print('b3 has fitness {}'.format(b3.fitness()))

b1 has fitness 23
b2 has fitness 21
b3 has fitness 22


# III. CrossOver and Mutation
Next, let's implement the **CrossOver** and **Mutation** step

In [22]:
def crossover(parent1, parent2):
    '''
    We implement cross-over where each pair of parents produce a child (a bit different with above description)
    '''
    c = np.random.randint(parent1.length)
    child_seq = np.append(parent1.sequences[:c], parent2.sequences[c:])
    return Board(parent1.length, child_seq)

def mutate(board):
    '''
    we mutate the board in-place
    '''
    i = np.random.randint(board.length)
    board.mutate(i)

Let's try our code on some example

In [23]:
c1 = crossover(b1, b2)
print('Child of b1 & b2\n\t{}'.format(c1.sequences))
mutate(c1)
print('After mutated, child c1 becomes\n\t{}'.format(c1.sequences))

Child of b1 & b2
	[3 2 7 4 8 5 5 4]
After mutated, child c1 becomes
	[3 2 7 4 8 5 5 4]


# IV. Population and GA
Now, the last step is to generate a population and reproduce (crossover/mutation) until we find a solution. We represent it in the class `Population`

In [75]:
class Population:
    def __init__(self, pop_size=100, board_size=8, fitness_eps=1e-5):
        self._pop_size   = pop_size
        self._board_size = board_size
        self._max_ap     = self._board_size * (self._board_size - 1) // 2
        self._fitness_eps= fitness_eps
        
        # randomly generate a population
        self._pop = []
        for i in range(self._pop_size):
            self._pop.append(Board(self._board_size))
        
        # compute the population fitness distribution
        self._fitness = np.zeros(self._pop_size)
        self.update_fitness()
    
    def update_fitness(self):
        for i in range(self._pop_size):
            self._fitness[i] = self._pop[i].fitness()
        
        # scaled to make it a distribution
        sum_fitness = np.sum(self._fitness)
        self._fitness /= sum_fitness
    
    def reproduce(self):
        new_pop = []
        for i in range(self._pop_size):
            parents = np.random.choice(self._pop_size, 2, replace=False, p = self._fitness)
            child = crossover(self._pop[parents[0]], self._pop[parents[1]])            
            new_pop.append(child)
        
        # update population
        self._pop = new_pop
        self.update_fitness()
        
        # we mutate a child if it's fitness < eps
        for i in range(self._pop_size):
            if self._fitness[i] < self._fitness_eps:
                mutate(self._pop[i])
        
        # update fitness
        self.update_fitness()
        
    def has_solution(self):
        for i in range(self._pop_size):
            if (self._pop[i].fitness() == self._max_ap):
                return self._pop[i]
        return None

def make_row(rowdata, col, empty, full):
    items = [col] * (2*len(rowdata) + 1)
    items[1::2] = (full if d else empty for d in rowdata)
    return ''.join(items)

def make_board(queens, col="|", row="---", empty="   ", full=" X "):
    size = len(queens)
    bar = make_row(queens, col, row, row)
    board = [bar] * (2*size + 1)
    board[1::2] = (make_row([i==q for i in range(size)], col, empty, full) for q in queens)
    return '\n'.join(board)
    
    
def genetic_algorithm(pop_size, board_size, fitness_eps=1e-5, max_iters=50):
    pop = Population(pop_size, board_size)
    sol = None
    for i in range(max_iters):
        pop.reproduce()
        
        sol = pop.has_solution()
        if sol is not None:
            print('At iteration {}, we found a board-solution\n\t{}'.format(i, sol.sequences))
            print('The board is')
            print(make_board(sol.sequences-1))
            break
    
    if sol is None:
        print('Reached max iteration {} NO SOLUTION FOUND!!!'.format(max_iters))

Let's test our GA on 8x8 board

In [76]:
genetic_algorithm(1000, 8, 1e-4, max_iters=100)

At iteration 33, we found a board-solution
	[4 8 5 3 1 7 2 6]
The board is
|---|---|---|---|---|---|---|---|
|   |   |   | X |   |   |   |   |
|---|---|---|---|---|---|---|---|
|   |   |   |   |   |   |   | X |
|---|---|---|---|---|---|---|---|
|   |   |   |   | X |   |   |   |
|---|---|---|---|---|---|---|---|
|   |   | X |   |   |   |   |   |
|---|---|---|---|---|---|---|---|
| X |   |   |   |   |   |   |   |
|---|---|---|---|---|---|---|---|
|   |   |   |   |   |   | X |   |
|---|---|---|---|---|---|---|---|
|   | X |   |   |   |   |   |   |
|---|---|---|---|---|---|---|---|
|   |   |   |   |   | X |   |   |
|---|---|---|---|---|---|---|---|


The GA works fine and it can find solution for 8x8 board, however it's still a bit slow. Can we tune it to make it to run faster?